In [109]:
import matplotlib. pyplot as plt
import tensorflow.keras
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array


#preprocessing required
from urllib import request

## Preprocessing of images

In [110]:
imgs_path = os.path.join('..', 'img')
img_size = (150, 150)

valid_images = [".jpg",".png",".jpeg",".JPG"]
data = []
labels = []

for root, dirs, files in os.walk(imgs_path):
    for filename in files:
        end = os.path.splitext(filename)[1]
        if end.lower() not in valid_images:
            continue
        image = load_img(os.path.join(root, filename), target_size=img_size)
        image = img_to_array(image)
        
        label = os.path.join(root, filename).split(os.path.sep)[-2]
        
        data.append(image)
        labels.append(label)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [111]:
# labels text to index
label_str = []

for i in range(len(labels)):
    try:
        j = label_str.index(labels[i])
    except:
        label_str.append(labels[i])
        j = label_str.index(labels[i])
    labels[i] = j

labels.astype(int)
#print(labels)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [112]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.2, random_state=3)

In [113]:
num_classes = 3

trainY = utils.to_categorical(trainY, num_classes)
testY = utils.to_categorical(testY, num_classes)

In [114]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_batches = datagen.flow(trainX, trainY, batch_size=32,
         subset='training')

test_batches = datagen.flow(trainX, trainY,
         batch_size=32, subset='validation')

train_path='Mask_Datasets\Train'
test_path='Mask_Datasets\Validation'


train_batches=ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_path,
        color_mode='grayscale',
        target_size=(150, 150),
        shuffle=True,
        classes=label_str)
            
test_batches=ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_path,
        target_size=(150, 150),
        shuffle=True,
        color_mode='grayscale',
        classes=label_str)            
            

# Keras Model

In [115]:
epochs=30
width,height=150,150
num_features=64


model=Sequential()

model.add(Conv2D(num_features,(3,3),activation='relu',input_shape=(width,height,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(120,activation='relu'))
model.add(Dense(3,activation='softmax'))


model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_104 (Conv2D)          (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_80 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_106 (Conv2D)          (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_107 (Conv2D)          (None, 15, 15, 128)     

In [116]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

In [117]:
history=model.fit(train_batches,epochs=15,
                           validation_data=test_batches,verbose=1,shuffle=True)

Epoch 1/15
8/8 [==============================] - 122s 2s/step - loss: 20.2581 - accuracy: 0.3755 - val_loss: 1.2113 - val_accuracy: 0.3279
Epoch 2/15
8/8 [==============================] - 10s 1s/step - loss: 1.0535 - accuracy: 0.4939 - val_loss: 0.9596 - val_accuracy: 0.5246
Epoch 3/15
8/8 [==============================] - 9s 1s/step - loss: 0.9435 - accuracy: 0.5673 - val_loss: 0.8065 - val_accuracy: 0.6393
Epoch 4/15
8/8 [==============================] - 9s 1s/step - loss: 0.7067 - accuracy: 0.6939 - val_loss: 0.5065 - val_accuracy: 0.8361
Epoch 5/15
8/8 [==============================] - 10s 1s/step - loss: 0.6006 - accuracy: 0.7306 - val_loss: 0.9535 - val_accuracy: 0.6066
Epoch 6/15
8/8 [==============================] - 12s 1s/step - loss: 0.7298 - accuracy: 0.6776 - val_loss: 0.5696 - val_accuracy: 0.7377
Epoch 7/15
8/8 [==============================] - 12s 1s/step - loss: 0.5620 - accuracy: 0.7551 - val_loss: 0.5095 - val_accuracy: 0.7705
Epoch 8/15
8/8 [==================

In [118]:
from tensorflow.keras.models import load_model

model.save('mask_three.h5')

In [119]:
#model = load_model('mask.h5')


In [120]:
pred=model.predict(test_batches)

In [121]:
np.argmax(model.predict(test_batches[0][0])[0])

0

In [122]:
predictions=model.predict_generator(test_batches, steps=28, verbose=1)
print(classification_report(test_batches[1], predictions))

28/28 [==============================] - 2s 28ms/step


ValueError: Found input variables with inconsistent numbers of samples: [2, 61]

In [ ]:
model.evaluate(pred,test_batches)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'tensorflow.python.keras.preprocessing.image.NumpyArrayIterator'>

In [ ]:
results = model.evaluate(testX, testY)
print(results)

2/2 [==============================] - 1s 485ms/step - loss: 0.1090 - accuracy: 0.9492
[0.10898660868406296, 0.9491525292396545]


In [ ]:
y_pred = model.predict(val_ds)
predicted_categories = np.argmax(y_pred, axis=1)

true_categories = tensorflow.concat([y for x, y in testX], axis=0).numpy()
true_categories_argmax = np.argmax(true_categories, axis=1)


ValueError: too many values to unpack (expected 2)